In [2]:
import subprocess
import re
import undetected_chromedriver as uc
import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

try:
    result = subprocess.run(
        [r'C:\Program Files\Google\Chrome\Application\chrome.exe', '--version'],
        stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
    )
    chrome_version = result.stdout.strip()
    if not chrome_version:
        print("오류")
except Exception as e:
    print(f"오류 발생: {e}")

In [3]:
from web3 import Web3

berachain_rpc_url = "https://bartio.rpc.berachain.com/"
chain_id = 80084

w3 = Web3(Web3.HTTPProvider(berachain_rpc_url))

try:
    if w3.is_connected():
        network_id = w3.eth.chain_id
        if network_id == chain_id:
            print(f"Berachain 연결 성공 (Chain ID: {network_id})")
        else:
            print(f"Chain ID 일치 X")
    else:
        print("Berachain 네트워크 연결 불가")
except Exception as e:
    print(f"오류 {e}")

Berachain 연결 성공 (Chain ID: 80084)


In [10]:
import requests
import json
import concurrent.futures

with open('./ABI/BeraChef.json') as f:
    BC_abi = json.load(f)

bera_chef = "0xfb81E39E3970076ab2693fA5C45A07Cc724C93c2"
BC_contract = w3.eth.contract(address=bera_chef, abi=BC_abi)

validator_url = "https://bartio-pol-indexer.berachain.com/berachain/v1alpha1/beacon/validators"
total_pages = 21
page_size = 10

Validator = []
AmountQueued = []
BGT_delegated = []
Commission = []
Reward_Rate = []
APY = []
Voting_Power = []
Lifetime_BGT = []
Lifetime_Incentives = []
Lifetime_HoneyValueBGT = []
Token_Count = []
Cutting_Board = []

def fetch(page):
    try:
        params = {
            "sortBy": "votingpower",
            "sortOrder": "desc",
            "page": page,
            "pageSize": page_size,
            "query": ""
        }
        response = requests.get(validator_url, params=params)

        if response.status_code == 200:
            data = response.json()
            validators = data.get("validators", [])

            page_results = []

            for v in validators:
                validator = {
                    "coinbase": v.get("coinbase"),
                    "amountStaked": v.get("amountStaked"),
                    "amountQueued": v.get("amountQueued"),
                    "commission": v.get("commission"),
                    "rewardRate": v.get("rewardRate"),
                    "apy": v.get("apy"),
                    "votingPower": v.get("votingPower"),
                    "allTimeBgtDirected": v.get("allTimeData", {}).get("allTimeBgtDirected"),
                    "allTimeIncentives": v.get("allTimeData", {}).get("allTimeHoneyValueTokenRewards"),
                    "alltimeHoneyValueBgtDirected": v.get("allTimeData", {}).get("alltimeHoneyValueBgtDirected"),
                    "tokenCount": v.get("allTimeData", {}).get("allTimeUniqueTokenCount"),
                    "cuttingBoard": BC_contract.functions.getActiveCuttingBoard(v.get("coinbase")).call()
                }
                page_results.append(validator)
            
            return page_results
        else:
            print(f"오류 발생 (페이지 {page} p): {response.status_code}")
            return []

    except Exception as e:
        print(f"오류 발생 (페이지 {page}): {e}")
        return []

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    future_page = {executor.submit(fetch, page): page for page in range(1, total_pages + 1)}
    
    for future in concurrent.futures.as_completed(future_page):
        page = future_page[future]
        try:
            results = future.result()
            for result in results:
                Validator.append(result["coinbase"])
                BGT_delegated.append(result["amountStaked"])
                AmountQueued.append(result["amountQueued"])
                Commission.append(result["commission"])
                Reward_Rate.append(result["rewardRate"])
                APY.append(result["apy"])
                Voting_Power.append(result["votingPower"])
                Lifetime_BGT.append(result["allTimeBgtDirected"])
                Lifetime_Incentives.append(result["allTimeIncentives"])
                Lifetime_HoneyValueBGT.append(result["alltimeHoneyValueBgtDirected"])
                Token_Count.append(result["tokenCount"])
                Cutting_Board.append(result["cuttingBoard"])
        except Exception as e:
            print(f"오류 발생 (페이지 {page} 처리 중): {e}")

In [12]:
import pandas as pd

validators = {
    "Validator": Validator,
    "BGT delegated": BGT_delegated,
    "Amount Queued": AmountQueued,
    "Cutting Board": Cutting_Board,
    "Commission": Commission,
    "Reward_Rate": Reward_Rate,
    "APY": APY,
    "Voting Power": Voting_Power,
    "Lifetime_BGT": Lifetime_BGT,
    "Lifetime_Incentives": Lifetime_Incentives,
    "Lifetime_HoneyValueBGT" : Lifetime_HoneyValueBGT,
    "Token Count": Token_Count
}

validator_info = pd.DataFrame(validators)

print(validator_info)

                                      Validator  BGT delegated  Amount Queued  \
0    0x450f6A64c1DB2e8EF127a6F1b5a5EE2c0414D793  236893.318093    4137.130497   
1    0xBBBBD949408332CB50B9f3cb47460b9Faa068758  233037.385088   13739.449883   
2    0xA7B927Cc85bFB997765aC86a86e46d36413e9B57  231046.945089    1679.363747   
3    0x1Cc335D9c67a71C777282fdb28b0a2d5eBf42AF4  228825.534662    4774.530844   
4    0x3ca67Ad85620D15354d2358dBb538b3fCB8F9b87  226700.938317    3592.294406   
..                                          ...            ...            ...   
204  0x760B5898079046E8Daa0059d3A3080fF28EBC3f0      45.508389      21.669573   
205  0xFa174BD57b20f0EE338b9699391B9f49f767FbEd      45.161131      17.363013   
206  0xF9fA3564BC56Dd5B1307B98000447419FC4A3d0a      43.682553       6.173055   
207  0x14eA75C0e1aB985A9594163D27C802D36f8c1497      42.946789      15.140253   
208  0x069da50b99408c8c42d006AfbF3C7F600384edEA      19.560912      18.299428   

                           

In [18]:
with open('./ABI/BerachainRewardsVault.json') as f:
    RV_abi = json.load(f)

vault_url = "https://bartio-pol-indexer.berachain.com/berachain/v1alpha1/beacon/vaults"
total_pages = 1
page_size = 10

LP = []
Product = []
VaultAddress = []
StakingToken = []
AmountStaked = []
AmountLeft = []
IncentiveRate = []
VaultWhiteList = []
WhiteListToken = []
TotalIncentive = []
Active_Vals = []
Active_ValCount = []
BGTCapture = []
TotalBgtReceived = []
VaultTotalSupply = []
RewardPerToken = []
RewardForDuration = []

def fetch_vault(page):
    try:
        params = {
            "sortBy": "activeIncentivesInHoney",
            "sortOrder": "desc",
            "page": page,
            "filterByProduct": "Kodiak,BEX",
            "pageSize": page_size,
            "query": ""
        }
        response = requests.get(vault_url, params=params)
       
        if response.status_code == 200:
            data = response.json()      
            vaults = data.get("vaults", [])

            page_results = []

            for v in vaults:
                vault = {
                    "vaultAddress": v.get("vaultAddress"),
                    "stakingTokenAddress": v.get("stakingTokenAddress"),
                    "amountStaked": v.get("amountStaked"),

                    "amountLeft": (v.get("activeIncentives", [{}])[0].get("amountLeft") if len(v.get("activeIncentives", [])) > 0 else None),
                    "incentiveRate": (v.get("activeIncentives", [{}])[0].get("incentiveRate") if len(v.get("activeIncentives", [])) > 0 else None),
                    "name":  v.get("metadata", {}).get("name"),
                    "product": v.get("metadata", {}).get("product"), 
                    "vaultWhitelist": v.get("vaultWhitelist"),    
                    "activeIncentivesInHoney": v.get("activeIncentivesInHoney"),
                    "activeValidators": v.get("activeValidators"),
                    "activeValidatorsCount": v.get("activeValidatorsCount"),
                    "bgtInflationCapture": v.get("bgtInflationCapture"),
                    "totalBgtReceived": v.get("totalBgtReceived")
                }            
                page_results.append(vault)
            
            return page_results
        else:
            print(f"오류 발생 (페이지 {page} p): {response.status_code}")
            return []

    except Exception as e:
        print(f"오류 발생 (페이지 {page}): {e}")
        return []

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    future_page = {executor.submit(fetch_vault, page): page for page in range(1, total_pages + 1)}
    
    for future in concurrent.futures.as_completed(future_page):
        page = future_page[future]
        try:
            results = future.result()
            for r in results:
                LP.append(r["name"])
                Product.append(r["product"])
                VaultAddress.append(r["vaultAddress"])
                StakingToken.append(r["stakingTokenAddress"])
                AmountStaked.append(r["amountStaked"])
                AmountLeft.append(r["amountLeft"])
                IncentiveRate.append(r["incentiveRate"])
                TotalIncentive.append(r["activeIncentivesInHoney"])
                Active_Vals.append(r["activeValidators"])
                Active_ValCount.append(r["activeValidatorsCount"])
                BGTCapture.append(r["bgtInflationCapture"])
                TotalBgtReceived.append(r["totalBgtReceived"])

                RV_contract = w3.eth.contract(address=r["vaultAddress"], abi=RV_abi)
                RewardForDuration.append(RV_contract.functions.getRewardForDuration().call())
                VaultTotalSupply.append(RV_contract.functions.totalSupply().call())
                WhiteListToken.append(RV_contract.functions.getWhitelistedTokens().call())
                RewardPerToken.append(RV_contract.functions.rewardPerToken().call())
        except Exception as e:
            print(f"오류 발생 (페이지 {page} 처리 중): {e}")

In [19]:
vaults = {
    "LP Token": LP,
    "Product": Product,
    "Reward Vault Address": VaultAddress,
    "Staking Token Address": StakingToken,
    "Total Supply Amount": VaultTotalSupply,
    "Reward For Duration": RewardForDuration,
    "Reward Per Token": RewardPerToken,
    "White List Token": WhiteListToken,
    
    "Total Incentive": TotalIncentive,
    "Amount Staked": AmountStaked,
    "Amount Left": AmountLeft,
    "Incentive Rate": IncentiveRate,
    "BGT Capture": BGTCapture,
    "Total Bgt Received": TotalBgtReceived,
    "Active_Validators": Active_Vals,
    "Active_Validators Count": Active_ValCount
}

vault_info = pd.DataFrame(vaults)

print(vault_info)

      LP Token Product                        Reward Vault Address  \
0   YEET-WBERA  Kodiak  0x175e2429bCb92643255abCbCDF47Fff63F7990CC   
1   HONEY-USDC  Kodiak  0xe5519D97eA854291c35a494b28929fA7abEf12e8   
2    PAW-HONEY     BEX  0x1992b26E2617928966B4F8e8eeCF41C6e7A77010   
3   HONEY-WETH     BEX  0x8288Ab28D6196cA4E89269B8644A146E493E23B3   
4   HONEY-USDC     BEX  0xe3b9B72ba027FD6c514C0e5BA075Ac9c77C23Afa   
5   iBGT-WBERA  Kodiak  0x7b15eeC57C60f8B68dF2b143c2CA5a772E787e86   
6  uniBTC-WBTC  Kodiak  0xEF55f90b05f719a2760A70cB1462Ff00B90E28C1   
7  HONEY-WBERA     BEX  0xAD57d7d39a487C04a44D3522b910421888Fb9C6d   
8   NECT-HONEY  Kodiak  0x5693809338Ae09481B5Da3A916d1cD1193F1B99A   

                        Staking Token Address          Total Supply Amount  \
0  0xE5A2ab5D2fb268E5fF43A5564e44c3309609aFF9    6505196229963022670684387   
1  0xb73deE52F38539bA854979eab6342A60dD4C8c03         99828778777105386413   
2  0xa51afAF359d044F8e56fE74B9575f23142cD4B76   11788802109095938

In [20]:
defaultTokenList_url = "https://raw.githubusercontent.com/berachain/default-lists/main/src/tokens/bartio/defaultTokenList.json"

response = requests.get(defaultTokenList_url)

Symbol = []
Name = []
ChainId = []
Address = []

if response.status_code == 200:
    data = response.json()
    tokens = data.get("tokens", [])

    for t in tokens:
        Symbol.append(t.get("symbol")),
        Name.append(t.get("name")),
        ChainId.append(t.get("chainId")),
        Address.append(t.get("address"))

defaultTokenLists = {
    "Name" : Name,
    "Symbol" : Symbol,
    "chainId": ChainId,
    "address": Address
}

defaultTokenLists_info = pd.DataFrame(defaultTokenLists)

print(defaultTokenLists_info)

                          Name   Symbol  chainId  \
0                  Bulla token    BULLA    80084   
1           Liquid Staked Buds   stBUDS    80086   
2                         Buds     BUDS    80086   
3           webera-vault-honey  weHONEY    80084   
4                          Paw      PAW    80084   
5                   Liquid BGT     LBGT    80084   
6                         YEET     YEET    80084   
7      Wrapped Berachain Token    WBERA    80084   
8              Berachain Token     BERA    80084   
9        Bera Governance Token      BGT    80084   
10                       Honey    HONEY    80084   
11  Stable Collateral USD Coin  STGUSDC    80084   
12             Tether USD Coin     USDT    80084   
13      Decentralized USD Coin      DAI    80084   
14                        WBTC     WBTC    80084   
15                        WETH     WETH    80084   
16   Honey Variable Debt Token  vdHONEY    80084   
17                Honey aToken   aHONEY    80084   
18          

In [21]:
import requests

url = "https://api.goldsky.com/api/public/project_clq1h5ct0g4a201x18tfte5iv/subgraphs/bgt-subgraph/v1000000/gn"

id_list = [
    "0xd69adb6fb5fd6d06e6ceec5405d95a37f96e3b96",
    "0xabc123456789abcdef123456789abcdef12345678",
    "0xdef987654321abcdef987654321abcdef98765432"
]

results = []

for addr in StakingToken:
    query = f"""
    query MyQuery {{
      poolShareAddress(id: "{addr.lower()}")
      {{
        id
        pool {{
          blockCreate
          tvlUsd
          template {{
            feeRate
          }}
          baseInfo {{
            id
            symbol
            beraValue
            usdValue
          }}
          baseAmount
          quoteInfo {{
            id
            symbol
            beraValue
            usdValue
          }}
          quoteAmount
          latestPoolDayData {{
            date
            tvlUsd
            volumeUsd
            feesUsd
          }}
        }}
      }}
    }}
    """
    
    response = requests.post(url, json={"query": query})
    
    if response.status_code == 200:
        data = response.json()
        results.append(data)
    else:
        print(f"Query failed for ID {id_value} with status code {response.status_code}")

for result in results:
    print(result)

{'data': {'poolShareAddress': None}}
{'data': {'poolShareAddress': None}}
{'data': {'poolShareAddress': {'id': '0xa51afaf359d044f8e56fe74b9575f23142cd4b76', 'pool': {'blockCreate': '3127699', 'tvlUsd': '129964791.913047210150105604', 'template': {'feeRate': 3000}, 'baseInfo': {'id': '0x0e4aaf1351de4c0264c5c7056ef3777b41bd8e03', 'symbol': 'HONEY', 'beraValue': '0.05430933298749375925635726601465248', 'usdValue': '1'}, 'baseAmount': '64982395.956523605075052802', 'quoteInfo': {'id': '0xb43fd1dc4f02d81f962e98203b2cc4fd9e342964', 'symbol': 'PAW', 'beraValue': '1.863533131865987707999005995562139', 'usdValue': '34.09894553022268715050066011696787'}, 'quoteAmount': '2198329.420363387485263449', 'latestPoolDayData': {'date': 1731801600, 'tvlUsd': '129964791.913047210150105604', 'volumeUsd': '490765.393386405729413616', 'feesUsd': '1472.296180159217188240848'}}}}}
{'data': {'poolShareAddress': {'id': '0xfbe71d98f9d2c658d52a2d72994c717637c3dda1', 'pool': {'blockCreate': '2132348', 'tvlUsd': '42